In [ ]:
# instalacja paczki związanej z narzędziem do analizy flow wykorzystywanym w dalszej części
# wykonanie tej komórki może chwilę potrwać
# w zależności od systemu i wersji obecnych paczek, instalacja może być utrudniona ze względu na konieczność rozwiązania różnych zależności
# przetestowano dla Google Colab
# Uwaga: paczka może mieć ograniczoną kompatybilność z Pythonem w wersji 3.12 - może być potrzeba przejść np. do 3.11

!pip install nfstream

In [ ]:
# import potrzebnych bibliotek do środowiska programistycznego (zostaną wykorzystane później)

import nfstream
from nfstream import NFStreamer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


## Pobranie i wczytanie danych do programu

In [ ]:
# Pobranie próbek z logami z sieci (w normalnym wypadku bralibyśmy je najpewniej gdzieś z naszej sieci)
!wget -O normal_traffic.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Normal-13/2017-07-03_capture-win2.pcap
!wget -O malicious_traffic.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-14/2013-10-18_capture-win15.pcap


# Wczytanie ruchu normalnego
normal_streamer = NFStreamer(source="normal_traffic.pcap", statistical_analysis = True)
normal_flows = normal_streamer.to_pandas()
normal_flows['label'] = 0  # Oznaczenie ruchu jako normalnego

# Wczytanie ruchu złośliwego
malicious_streamer = NFStreamer(source="malicious_traffic.pcap", statistical_analysis = True)
malicious_flows = malicious_streamer.to_pandas()
malicious_flows['label'] = 1  # Oznaczenie ruchu jako złośliwego

# Połączenie obu zbiorów danych
data = pd.concat([normal_flows, malicious_flows], ignore_index=True)

# Wyświetlenie pierwszych rekordów zbiorów danych
data.head()


## Eksploracja i czyszczenie danych

In [ ]:
# Wyświetlenie podstawowych informacji o zbiorach danych
data.info()

# Usuwanie kolumn, które mają tylko jedną unikalną wartość lub brakujące wartości
for col in data.columns:
    if data[col].nunique() == 1 or data[col].isnull().any():
        data.drop(col, inplace=True, axis=1)

# Wyświetlenie liczby rekordów po czyszczeniu
print(f"Liczba rekordów po czyszczeniu: {data.shape[0]}")


In [ ]:
# Eksperymenty - eksploracja szczegółowa

# Wyświetlenie unikalnych wartości w kolumnach (można wybrać kolumnę do analizy)
column_name = 'src_ip'  # Przykład nazwy kolumny
print("Unikalne wartości w kolumnie:", column_name)
print(data[column_name].unique())

# Eksploracja liczby rekordów dla każdej etykiety
print("Liczba rekordów dla każdej etykiety:")
print(data['label'].value_counts())

# Histogramy dla wybranych cech numerycznych (należy odkomentować i zmodyfikować)
# data['some_numeric_column'].hist(bins=30)
# plt.title('Histogram cechy some_numeric_column')
# plt.show()


## Analiza korelacji cech

In [ ]:
import seaborn as sns

# Wybranie tylko kolumn numerycznych (int i float)
data = data.select_dtypes(include=[np.number])

# Sprawdzenie, czy kolumna 'label' istnieje w danych
if 'label' in data.columns:
    # Obliczanie korelacji wszystkich cech z etykietą
    correlation_with_label = data.corr()['label'].sort_values(key=abs, ascending=False)

    # Usunięcie korelacji etykiety z samą sobą
    correlation_with_label = correlation_with_label.drop('label', errors='ignore')

    # Wybór 10 najbardziej skorelowanych cech
    top_10_correlated_features = correlation_with_label.head(10)

    print("10 cech najbardziej skorelowanych z etykietą:\n", top_10_correlated_features)

    # Wizualizacja korelacji tych cech z etykietą
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_10_correlated_features.index, y=top_10_correlated_features.values, palette='viridis')
    plt.title('Top 10 cech najbardziej skorelowanych z etykietą')
    plt.xlabel('Cechy')
    plt.ylabel('Współczynnik korelacji')
    plt.xticks(rotation=45)  # Obrót etykiet osi X dla lepszej czytelności
    plt.show()
else:
    print("Brak kolumny 'label' w danych.")


In [ ]:
# Eksperymenty - analiza wybranych korelacji

# Wybór korelacji do szczegółowej analizy (należy odkomentować i zmodyfikować na atrybuty które chcemy sprawdzić)
# feature1 = 'source_port'
# feature2 = 'destination_port'
# sns.scatterplot(x=data[feature1], y=data[feature2])
# plt.title(f'Korelacja między {feature1} a {feature2}')
# plt.show()


## Przygotowanie danych do modelowania

In [ ]:
X = data.drop('label', axis=1)
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Budowanie i trenowanie modelu

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

def train_and_evaluate_decision_tree(X_train, y_train, X_test, y_test, max_depth=None, criterion='gini'):
    # Trenowanie modelu Decision Tree z określonymi parametrami
    tree_model = DecisionTreeClassifier(max_depth=max_depth, criterion=criterion, random_state=42)
    tree_model.fit(X_train, y_train)

    # Przewidywanie na zestawie testowym
    predictions = tree_model.predict(X_test)

    # Obliczenie metryk
    accuracy = accuracy_score(y_test, predictions)
    conf_matrix = confusion_matrix(y_test, predictions)

    # Wizualizacja drzewa
    plt.figure(figsize=(20,10))
    plot_tree(tree_model, filled=True, feature_names=X_train.columns, class_names=['Normal', 'Malicious'], fontsize=10)
    plt.title("Wizualizacja drzewa decyzyjnego")
    plt.show()

    return tree_model, accuracy, conf_matrix


In [ ]:
# Trenowanie i ocena modelu z domyślnymi parametrami
default_model, default_accuracy, default_conf_matrix = train_and_evaluate_decision_tree(X_train, y_train, X_test, y_test)

print("Dokładność modelu z domyślnymi parametrami:", default_accuracy)
sns.heatmap(default_conf_matrix, annot=True, fmt="d")
plt.title("Macierz błędów - Model domyślny")
plt.show()


## Interaktywne testowanie parametrów modelu

In [ ]:
from ipywidgets import interactive

def interactive_decision_tree_testing(max_depth, criterion):
    _, new_accuracy, new_conf_matrix = train_and_evaluate_decision_tree(X_train, y_train, X_test, y_test, max_depth, criterion)

    print("Dokładność modelu na nowym zestawie danych:", new_accuracy)
    sns.heatmap(new_conf_matrix, annot=True, fmt="d", cmap='viridis')
    plt.title("Macierz błędów na nowym zestawie danych")
    plt.show()

# Stworzenie interaktywnego widgetu do eksperymentowania z parametrami
interactive_plot = interactive(interactive_decision_tree_testing,
                               max_depth=(1, 20, 1),
                               criterion=["gini", "entropy"])
interactive_plot


Dla pokazanego przykładu można zaobserwować, że pomimo zwiększania ogranicznenia dotyczącego głębokości drzewa, kontrykcja się już nie zmienia, ponieważ przy niższym poziomie głebokości model przestaje się już kompletnie mylić.

**Zadanie:** Zamodeluj bardziej złożoną sytuację dobierając inne zbiory danych, można wybierać z tego samego repozytorium: https://mcfp.felk.cvut.cz/publicDatasets/ (uwaga na rozmiary zbiorów, dla dużych objętności będzie to trwało bardzo długo). W tym celu najlepiej przerabiać kod zaprezentowanny powyżej, albo przeklejać komórki na nowo poniżej i edytować je w nowym miejscu.